In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/Adadelta/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adadelta/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_1/kernel/read)]]
	 [[{{node loss/mul/_93}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_835_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [10]:
with tf.Graph().as_default():
  module_url = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1"
  embed = hub.Module(module_url)
  embeddings = embed(["A long sentence.", "single-word",
                      "http://example.com"])

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

    print(sess.run(embeddings))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[[ 1.27072588e-01  1.70752838e-01 -9.90944505e-02  7.04903603e-02
   2.26204265e-02 -8.07912350e-02  1.34882599e-01  9.98359919e-02
  -9.38840210e-02  1.57970995e-01  1.49555340e-01 -7.23052993e-02
   2.82968562e-02 -7.33467517e-03  7.44888186e-02 -2.56097335e-02
   1.71663105e-01 -2.87602507e-02  5.61525524e-02  1.13487214e-01
  -1.64960790e-02 -6.53510634e-03 -2.56301146e-02 -8.42670649e-02
   8.53472874e-02 -5.40463366e-02  7.97923282e-02 -1.38562024e-01
  -1.34141564e-01 -7.57983997e-02 -9.42738429e-02 -7.67866522e-02
  -1.52945921e-01  1.80598632e-01  1.54585212e-01 -1.51075236e-02
   3.27526703e-02  9.76694282e-03 -5.50296791e-02  6.87522218e-02
  -4.60033529e-02 -6.41337559e-02  9.67478678e-02  2.06821620e-01
  -3.41980383e-02  1.59218937e-01 -1.25459984e-01  1.11098640e-01
  -8.97483621e-03  7.16845766e-02  9.47824046e-02 -1.14044234e-01
  -2.18231454e-01 -4.69759107e-03 -2.76438929e-02  2.

In [4]:
tf.logging.set_verbosity("INFO")

In [5]:
# Declare the module
gan = hub.Module("https://tfhub.dev/google/compare_gan/model_15_cifar10_resnet_cifar/1")

# Use the generator signature
z_values = tf.random_uniform(minval=-1, maxval=1, shape=[64, 128])
images = gan(z_values, signature="generator")

# Use the discriminator signature
logits = gan(images, signature="discriminator")



INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/compare_gan/model_15_cifar10_resnet_cifar/1'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/compare_gan/model_15_cifar10_resnet_cifar/1'.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


NameError: name 'session' is not defined

In [8]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    # Drive execution with tf.Session
    session.run([images, logits])

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node module_apply_generator/generator_evaluation/generator/B1/up_conv_shortcut/Conv2D (defined at C:\dev\Miniconda3\lib\site-packages\tensorflow_hub\native_module.py:447)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](module_apply_generator/generator_evaluation/generator/B1/up_conv_shortcut/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, module_apply_generator/generator/B1/up_conv_shortcut/w/read)]]
	 [[{{node module_apply_generator/generator_evaluation/result/_439}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1041_module_apply_gene

In [9]:
gan = hub.Module("https://tfhub.dev/google/compare_gan/model_1_celebahq128_resnet19/1")

# Use the generator signature
z_values = tf.random_uniform(minval=-1, maxval=1, shape=[64, 128])
images = gan(z_values, signature="generator")

# Use the discriminator signature
logits = gan(images, signature="discriminator")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


FailedPreconditionError: Attempting to use uninitialized value module_2/generator/B3/bn1/gamma
	 [[node module_2_apply_generator/generator/B3/bn1/gamma/read (defined at C:\dev\Miniconda3\lib\site-packages\tensorflow_hub\native_module.py:447)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](module_2/generator/B3/bn1/gamma)]]
	 [[{{node module_2_apply_discriminator/discriminator_representation/result/_1}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_807_module_2_apply_discriminator/discriminator_representation/result", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'module_2_apply_generator/generator/B3/bn1/gamma/read', defined at:
  File "C:\dev\Miniconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\dev\Miniconda3\lib

In [10]:
with tf.Session() as sess:
# Drive execution with tf.Session
    sess.run(tf.global_variables_initializer())
    sess.run([images, logits])

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node module_2_apply_generator/generator_evaluation/generator/B1/up_conv_shortcut/Conv2D (defined at C:\dev\Miniconda3\lib\site-packages\tensorflow_hub\native_module.py:447)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](module_2_apply_generator/generator_evaluation/generator/B1/up_conv_shortcut/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, module_2_apply_generator/generator/B1/up_conv_shortcut/w/read)]]
	 [[{{node module_2_apply_discriminator/discriminator_representation/result/_1855}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8